<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Deepfake and Manipulated Media Analysis Data Download**

In [18]:
# Install required dependencies
!pip install -q datasets pandas pillow tqdm huggingface_hub decord
!apt-get install -y ffmpeg  # Install FFmpeg for video processing

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [19]:
import os
import pandas as pd
from pathlib import Path
from datasets import load_dataset
from tqdm import tqdm
from PIL import Image
from huggingface_hub import login
import getpass

In [20]:
# Authenticate Hugging Face
HF_TOKEN = getpass.getpass("Enter your Hugging Face token: ")
login(HF_TOKEN)

Enter your Hugging Face token: ··········


In [21]:
# Set download directory
BASE_DIR = Path.home() / "Downloads" / "deepfake_dataset"
BASE_DIR.mkdir(parents=True, exist_ok=True)
metadata = []

In [22]:
def download_data(dataset_name, category_name, file_ext, key):
    """Downloads and saves data (images, videos, audio)"""
    print(f"\n📥 Downloading {category_name} Data...")

    try:
        dataset = load_dataset(dataset_name, split="train", streaming=True)
    except Exception as e:
        print(f"❌ Error loading {dataset_name}: {e}")
        return

    save_dir = BASE_DIR / category_name
    save_dir.mkdir(parents=True, exist_ok=True)

    real_samples, fake_samples = [], []

    # Collect 40 samples and manually separate into 20 real & 20 fake
    for sample in dataset.take(40):
        if sample["label"] == 0 and len(real_samples) < 20:
            real_samples.append(sample)
        elif sample["label"] == 1 and len(fake_samples) < 20:
            fake_samples.append(sample)
        if len(real_samples) == 20 and len(fake_samples) == 20:
            break

    # Save real and fake samples
    for category, samples in zip(["real", "fake"], [real_samples, fake_samples]):
        (save_dir / category).mkdir(parents=True, exist_ok=True)

        for i, sample in enumerate(tqdm(samples, desc=f"Saving {category} {category_name}")):
            file_path = save_dir / category / f"{category}_{i:03d}.{file_ext}"
            try:
                # Handle both in-memory bytes and file paths
                if isinstance(sample[key], dict) and 'bytes' in sample[key]:
                    content = sample[key]['bytes']
                    with open(file_path, "wb") as f:
                        f.write(content)
                else:
                    os.system(f"wget {sample[key]} -O {file_path}")

                metadata.append({
                    "modality": category_name,
                    "category": category,
                    "filename": file_path.name,
                    "file_path": str(file_path),
                    "source_dataset": dataset_name
                })
            except Exception as e:
                print(f"❌ Error saving file {file_path}: {e}")

def save_metadata():
    """Saves metadata as CSV"""
    pd.DataFrame(metadata).to_csv(BASE_DIR / "metadata.csv", index=False)
    print("✅ Metadata saved")

def main():
    # Download datasets
    download_data("thenewsupercell/fixed-fakeavceleb", "audio", "wav", "audio")
    download_data("mkhLlamaLearn/dfdc", "videos", "mp4", "video")
    download_data("taohu/faceforensics_h5", "images", "jpg", "image")
    save_metadata()
    print(f"\n🎉 Dataset Collection Complete! Data stored in {BASE_DIR}")

In [23]:
if __name__ == "__main__":
    main()


📥 Downloading audio Data...


RuntimeError: Error reading /kaggle/working/new_video_dataset/train/FakeVideo-FakeAudio/00001_0_id06152_wavtolip.mp4...